<div style='color: #690027;' markdown="1">
    <h1>Detectie van stomata</h1> 
</div>

Voer onderstaande codecel uit om van de methodes in deze notebook gebruik te kunnen maken.

In [ ]:
import imp
with open('../.scripts/diep_neuraal_netwerk.py', 'rb') as fp:
    diep_neuraal_netwerk = imp.load_module('.scripts', fp, '../.scripts/diep_neuraal_netwerk.py', ('.py', 'rb', imp.PY_SOURCE))

<div style='color: #690027;' markdown="1">
    <h2>1. Het referentie model</h2> 
</div>

In deze notebook wordt een diep neuraal netwerk gebruikt om stomata op een grote afbeelding te herkennen. Hoe zo een netwerk werkt en wat de onderdelen zijn kan je terugvinden in [BasisDiepNeuraalNetwerk](../IntroductieDeepLearning/BasisDiepNeuraalNetwerk.ipynb). Het netwerk waarvoor we hebben gekozen gebruikt de convolutionele lagen van het bestaande netwerk VGG19, heeft 1 feedforward laag met 1024 neuronen en gebruikt 'adam' als optimizer (i.p.v. SGD) met een learning rate van 0,0001. Het netwerk gebruikt ook dezelfde regularisatie technieken die beschreven zijn in [Overfitting](../IntroductieDeepLearning/Overfitting.ipynb). Volgende afbeelding geeft dit netwerk weer zoals de netwerken die we gezien hebben in [BasisDiepNeuraalNetwerk](../IntroductieDeepLearning/BasisDiepNeuraalNetwerk.ipynb).

<img src="../.images/IntroductieDeepLearning/VGG19_netwerk.jpg"/>

Om dit netwerk in te laden voer je volgende codecel uit:

In [ ]:
diep_neuraal_netwerk.laad_referentie_model()

<div style='color: #690027;' markdown="1">
    <h2>2. Stomata detectie in een volledige afbeelding</h2> 
</div>

Momenteel kan het getrainde model een classificatie maken tussen 2 klassen ("wel een stoma" of "geen stoma") wanneer het een afbeelding van 120 pixels op 120 pixels krijgt als invoer. We willen echter dat alle stomata in een grote afbeelding (1600px op 1200px) gevonden kunnen worden met ons model. Om dit te verwezenlijken maken we gebruik van een <b>sliding window</b>. Dit is een vierkantje van 120px op 120px dat uit de grote afbeelding wordt geknipt en als invoer van het getrainde model wordt gebruikt. Voor dit vierkantje zal het model dan een voorspelling geven waarna het vierkantje naar rechts opschuift met 10 pixels. Wanneer het vierkantje de volledige breedte van de afbeelding heeft doorlopen zal het opnieuw naar de linkerkant verspringen maar dan 10 pixels lager dan de vorige keer. Dit proces herhaald zich tot de volledige afbeelding verwerkt werd met telkens sprongen van 10 pixels. Volgende afbeelding geeft een illustratie.

<img src="../.images/IntroductieDeepLearning/sliding_window.gif"/>

Voor elke vierkantje uit de grote afbeelding zal er op deze manier een voorspelling gemaakt worden. Vaak zullen er meerdere vierkantjes rond de effectieve stoma een positieve uitvoer geven ("wel een stoma") omdat er slechts met 10 pixels wordt opgeschoven en dus niet veel verschil in de vierkantjes zit. Om deze vele positieve voorspellingen samen te voegen wordt er gebruik gemaakt van <b>clustering</b>. Clustering zal punten die dicht bij elkaar liggen als 1 cluster beschouwen en het middelpunt van deze clusters zijn dan de gevonden stomata. Volgende afbeelding geeft een voorbeeld van clustering waarbij de kleuren de verschillende clusters voorstellen en de grote punten de centers van de clusters.

<img src="../.images/IntroductieDeepLearning/clustering.jpg" width="400"/>

Bij de detectie speelt de <b>drempelwaarde</b> ook een belangrijke rol. Deze drempelwaarde bepaalt voor welke waarden van de uitvoer we de invoer als een stoma beschouwen. Als we bijvoorbeeld 0.5 als drempelwaarde nemen zal alle uitvoer groter dan 0.5 als "wel een stoma" beschouwd worden en alle uitvoer kleiner dan 0.5 als "geen stoma". 

Een belangrijke afweging die je moet maken bij het kiezen van de drempelwaarde is de verhouding tussen <b>precision</b> en <b>recall</b>. 

<ul>
    <li>Precision: het percentage van de gevonden stomata dat ook daadwerkelijk stoma zijn.</li>
    <li>Recall: het percentage van de totale aanwezige stomata op een afbeelding die gevonden werden.</li>
</ul>

Het is duidelijk dat een lage drempelwaarde zal zorgen voor een lage precision (er worden meer punten al stomata beschouwd, dus ook slechte punten) maar een hoge recall (er gaan ook meer echte stomata gevonden worden). Andersom geldt dit ook, een hoge drempelwaarde zal zorgen voor een hoge precision maar lage recall.

Volgende codecel zal voor 3 afbeeldingen de stomata detecteren met het referentie netwerk. Je kan de drempelwaarde (thr) aanpassen en het resultaat interpreteren.

In [ ]:
diep_neuraal_netwerk.vind_stomata()

<div style='color: #690027;' markdown="1">
    <h2>3. Adversarial learning</h2> 
</div>

Vaak is het mogelijk een diep neuraal netwerk te misleiden door opzettelijk verkeerde invoer te geven, dit wordt ook wel adversarial learning genoemd. Als we ons netwerk trainen gaan we ervan uit dat de uiteindelijke invoer er ongeveer zo uitziet als de data waarmee het netwerk getraind werd. Maar wat als dit nu niet het geval is? Kan ons netwerk een onderscheid maken tussen een microscopische afbeelding van een blad en een andere afbeelding? 

Voer volgende codecel uit om te zien hoe het netwerk reageert als we een foto van een kat als invoer geven.

In [ ]:
diep_neuraal_netwerk.misleid_netwerk()

Je kan zien dat het netwerk niet echt leert wat een stoma is maar gewoon een patronen leert herkennen. De ogen van de kat hebben ongeveer dezelfde vorm als een stoma en zullen dus ook als stoma gedetecteerd worden.

In dit voorbeeld kreeg het netwerk een invoer die duidelijk verschilt van de gewenste invoer, maar dit is niet altijd het geval. Het is mogelijk een afbeelding zo te manipuleren dat het voor de mens dezelfde afbeelding lijkt maar een neuraal netwerk een totaal andere conclussie trekt. In een paper ([Explaining and Harnessing Adversarial Examples](https://arxiv.org/abs/1412.6572)) hebben enkele onderzoekers een afbeelding van een panda aangepast door ruiswaarden toe te voegen. Een diep neuraal netwerk dat afbeeldingen classificeerd was voor de aanpassing 57,7% zeker dat er een panda op de afbeelding stond, na de aanpassing dacht het netwerk met 99,3% zekerheid dat het om een gibbon ging.

<img src="../.images/IntroductieDeepLearning/adversarial_learning.jpg"/>

Deze techniek kan ook misbruikt worden, denk maar aan een zelfrijdende auto die verkeersborden herkent met een diep neuraal netwerk. Iemand met kwade bedoelingen zou een normaal stopbord kunnen vervangen door een aangepast stopbord dat er gelijkaardig uitziet voor de mens maar niet kan gedetecteerd worden door het neurale netwerk van de auto.

Andere voorbeelden zijn:
<ul>
    <li>Het aanpassen van spam emails om een spam filter te omzeilen.</li>
    <li>Het namaken van een foto om toegang te krijgen tot een GSM met gezichtsherkenning.</li>
    <li>...</li>
</ul>